In [1]:
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from PIL import Image

# Step 1: Create a Synthetic Dataset (if no dataset exists)
def create_synthetic_dataset(base_dir='hand_gesture_dataset', image_size=(64, 64), num_classes=3, num_images_per_class=50):
    os.makedirs(base_dir, exist_ok=True)
    class_names = [f'class_{i}' for i in range(num_classes)]
    for class_name in class_names:
        class_dir = os.path.join(base_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        for i in range(num_images_per_class):
            random_image = np.random.randint(0, 256, (image_size[0], image_size[1], 3), dtype=np.uint8)
            img = Image.fromarray(random_image)
            img.save(os.path.join(class_dir, f'{class_name}_{i}.jpg'))
    print(f"Synthetic dataset created at: {os.path.abspath(base_dir)}")
    return class_names

# Step 2: Load the Dataset
def load_images_from_folder(folder, image_size=(64, 64)):
    images = []
    labels = []
    class_names = os.listdir(folder)
    for label, class_name in enumerate(class_names):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(class_folder, filename)
                img = Image.open(img_path).resize(image_size)
                images.append(np.array(img))
                labels.append(label)
    return np.array(images), np.array(labels), class_names

# Step 3: Define the CNN Model
def build_cnn_model(input_shape, num_classes):
    model = Sequential([
        Input(shape=input_shape),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Step 4: Main Script
if __name__ == "__main__":
    # Dataset parameters
    dataset_path = 'hand_gesture_dataset'
    image_size = (64, 64)

    # Create synthetic dataset if it doesn't exist
    if not os.path.exists(dataset_path):
        class_names = create_synthetic_dataset(base_dir=dataset_path, image_size=image_size, num_classes=3, num_images_per_class=50)
    else:
        class_names = os.listdir(dataset_path)

    # Load the dataset
    X, y, class_names = load_images_from_folder(dataset_path, image_size=image_size)
    X = X / 255.0  # Normalize pixel values to [0, 1]
    y = to_categorical(y, num_classes=len(class_names))  # One-hot encode labels

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Data Augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    datagen.fit(X_train)

    # Build the CNN model
    model = build_cnn_model(input_shape=(image_size[0], image_size[1], 3), num_classes=len(class_names))

    # Train the model
    model.fit(datagen.flow(X_train, y_train, batch_size=16), epochs=10, validation_data=(X_test, y_test))

    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

    # Save the model
    model.save('hand_gesture_model.keras')
    print("Model saved as 'hand_gesture_model.keras'")

Synthetic dataset created at: /content/hand_gesture_dataset


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 347ms/step - accuracy: 0.3522 - loss: 1.1244 - val_accuracy: 0.3000 - val_loss: 1.1134
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - accuracy: 0.3564 - loss: 1.1124 - val_accuracy: 0.3000 - val_loss: 1.1038
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.2860 - loss: 1.1132 - val_accuracy: 0.3000 - val_loss: 1.1126
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 143ms/step - accuracy: 0.3621 - loss: 1.1013 - val_accuracy: 0.3000 - val_loss: 1.1011
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.2798 - loss: 1.1020 - val_accuracy: 0.3000 - val_loss: 1.0994
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.4748 - loss: 1.0942 - val_accuracy: 0.3000 - val_loss: 1.1049
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 137ms/step - accuracy: 0.3326 - loss: 1.1069 - val_accuracy: 0.3000 - val_loss: 1.1070
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 135ms/step - accuracy: 0.3309 - loss: 1.1038 - val_accuracy: 0.3000 - val_loss